In [1]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = bassamsabber
%env KAGGLE_KEY = 717678f8ebfb53d84e9c47444c275871
!kaggle competitions download -c computationalintelligencesc2020
!unzip DataSet
!unzip computationalintelligencesc2020

Streaming output truncated to the last 5000 lines.
  inflating: Scenes training set/Scenes training set/mountain/7539.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7551.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7560.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7565.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7578.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7581.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7586.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7647.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7652.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7654.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7662.jpg  
  inflating: Scenes training set/Scenes training set/mountain/767.jpg  
  inflating: Scenes training set/Scenes training set/mountain/7672.jpg  
 

In [1]:
import numpy as np
import cv2
from tqdm import tqdm 
import os
import pandas as pd
from os.path import join
from random import shuffle
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers, layers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Conv2D, SeparableConv2D, Add, Dense, BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D, Concatenate, Average,Maximum
from keras.utils.data_utils import get_file

print("Done")

Done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#seed
import numpy as np
import tensorflow as tf
import random as rn

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(42)

rn.seed(12345)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K


tf.compat.v1.set_random_seed(1234)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [5]:
v1 ="/content/Scenes training set/Scenes training set/buildings"
v2 = "/content/Scenes training set/Scenes training set/forest"
v3 = "/content/Scenes training set/Scenes training set/glacier"
v4 = "/content/Scenes training set/Scenes training set/mountain"
v5 = "/content/Scenes training set/Scenes training set/sea"
v6 = "/content/Scenes training set/Scenes training set/street"

IMG_SIZE = 224

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(v1)):
        path = os.path.join(v1, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([1,0,0,0,0,0])]) 
        
    for img in tqdm(os.listdir(v2)):
        path = os.path.join(v2, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,1,0,0,0,0])])
    
    for img in tqdm(os.listdir(v3)):
        path = os.path.join(v3, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,1,0,0,0])])

    for img in tqdm(os.listdir(v4)):
        path = os.path.join(v4, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,0,1,0,0])])

    for img in tqdm(os.listdir(v5)):
        path = os.path.join(v5, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,0,0,1,0])])
    for img in tqdm(os.listdir(v6)):
        path = os.path.join(v6, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,0,0,0,1])])

    shuffle(training_data)
    #train_data=np.load('/content/drive/My Drive/data/train_data.npy', allow_pickle=True) 
    np.save('train.npy', training_data)
    return training_data 


   
train_dataa = create_train_data() 

100%|██████████| 2382/2382 [00:02<00:00, 1092.36it/s]
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [9]:
TEST_DIR = "/content/Scenes testing test/Scenes testing test"
def process_test_data():
    testing_data = [] 
    for img in tqdm(os.listdir(TEST_DIR)):
        name=img
        path = os.path.join(TEST_DIR, img) 
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img),name])
    np.save('test.npy', testing_data) 
    return testing_data

test_data = process_test_data()

100%|██████████| 3050/3050 [00:04<00:00, 716.42it/s]
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
 train_data=np.load("/content/drive/MyDrive/natural scense/train.npy", allow_pickle=True) 
 test_data=np.load("/content/drive/MyDrive/natural scense/test.npy", allow_pickle=True) 

In [4]:
#Load weights of VGG19 NO TOP ON IMAGENET
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',WEIGHTS_PATH_NO_TOP)

In [5]:
def VGG19(include_top=False,weights='imagenet',
          classes=6):
		

		input = Input(shape=(224, 224, 3))

		x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(input)
		x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
		x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

		x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
		x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

		x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
		x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)


		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
		x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

		# x = Flatten()(x)
		# x = Dense(units=4096, activation='relu')(x)
		# # x = Dropout(0.5)(x)
		# x = Dense(units=4096, activation='relu')(x)
		# # x = Dropout(0.5)(x)
		# output = Dense(units=1000, activation='softmax')(x)

		model = Model(inputs=input, outputs=x)
		
		model.load_weights(weights_path)

		return model
 
model = VGG19(include_top=False, weights='imagenet', classes=6)

model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0     

In [6]:
model.trainable = False

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0     

In [8]:
new_model = model.output
global_layer = GlobalAveragePooling2D() 
new_model = global_layer(new_model)
new_model = Dense(3072, activation="relu")(new_model)
new_model = Dense(6, activation="softmax")(new_model)

In [10]:
vgg_model = Model(inputs=model.input, outputs=new_model)

In [11]:
vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0   

In [13]:
IMG_SIZE = 224

opt = optimizers.SGD(lr=.0001,decay=1e-6 ,momentum=0.9,nesterov=True)

vgg_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Xt = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 

Yt = np.asarray([i[1] for i in train_data])

vgg_model.fit(Xt, Yt, batch_size=8, epochs=20, verbose=1)
  
vgg_model.save("vgg19.model")

Epoch 1/20
1755/1755 [==============================] - 63s 31ms/step - loss: 0.4173 - accuracy: 0.8519
Epoch 2/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.2240 - accuracy: 0.9186
Epoch 3/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1946 - accuracy: 0.9318
Epoch 4/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1759 - accuracy: 0.9366
Epoch 5/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1703 - accuracy: 0.9399
Epoch 6/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1459 - accuracy: 0.9506
Epoch 7/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1392 - accuracy: 0.9510
Epoch 8/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1367 - accuracy: 0.9498
Epoch 9/20
1755/1755 [==============================] - 55s 31ms/step - loss: 0.1264 - accuracy: 0.9545
Epoch 10/20
1755/1755 [==============================] - 55s 31m

In [14]:
Submit=[]
for num, data in enumerate(test_data[:]): 
    img_data = data[0]
    name=data[1] 
    data = img_data.reshape(-1,224, 224, 3) 
    model_out = vgg_model.predict([data])[0]
    if np.argmax(model_out) == 0:
        Submit.append([name,0])
    elif np.argmax(model_out) == 1:
        Submit.append([name,1])
    elif np.argmax(model_out) == 2:
        Submit.append([name,2])
    elif np.argmax(model_out) == 3:
        Submit.append([name,3])
    elif np.argmax(model_out) == 4:
        Submit.append([name,4])
    elif np.argmax(model_out) == 5:
        Submit.append([name,5])                                 
print(Submit)
Submitt=pd.DataFrame(Submit,columns=['Image','Label'])
Submitt.to_csv("/content/submit.csv", index = False) 

[['23004.jpg', 3], ['23215.jpg', 5], ['20998.jpg', 2], ['20627.jpg', 2], ['23885.jpg', 4], ['23291.jpg', 3], ['22374.jpg', 4], ['20683.jpg', 0], ['22636.jpg', 3], ['22623.jpg', 4], ['20647.jpg', 4], ['21277.jpg', 4], ['22450.jpg', 5], ['24115.jpg', 3], ['21967.jpg', 2], ['20283.jpg', 4], ['23597.jpg', 1], ['20751.jpg', 2], ['22311.jpg', 5], ['24213.jpg', 2], ['22346.jpg', 3], ['22400.jpg', 1], ['22490.jpg', 1], ['21936.jpg', 4], ['24175.jpg', 5], ['20334.jpg', 3], ['23024.jpg', 3], ['22149.jpg', 1], ['21017.jpg', 0], ['22268.jpg', 1], ['24132.jpg', 4], ['24250.jpg', 5], ['23429.jpg', 3], ['21230.jpg', 2], ['546.jpg', 0], ['23932.jpg', 0], ['22097.jpg', 0], ['23419.jpg', 4], ['22724.jpg', 3], ['20807.jpg', 5], ['22064.jpg', 0], ['21287.jpg', 5], ['20261.jpg', 1], ['949.jpg', 0], ['21186.jpg', 3], ['22895.jpg', 5], ['23177.jpg', 2], ['23876.jpg', 1], ['20726.jpg', 2], ['24133.jpg', 2], ['22614.jpg', 3], ['21294.jpg', 0], ['20096.jpg', 0], ['22158.jpg', 3], ['20435.jpg', 3], ['24297.jpg',

In [ ]:
'''r = 0
w = 0
import csv

with open('/content/drive/MyDrive/scc/aaa/final2.csv', newline='') as f:
    reader = csv.reader(f)
    b1 = list(reader)
with open('/content/drive/MyDrive/scc/res152(csv)/res224.csv', newline='') as f:
    reader = csv.reader(f)
    b2 = list(reader)
for i in range(1,3051):
	if b1[i][1] == b2[i][1]:
		r+=1
w = 3050 - r
acc = (r/3050)*100
print(acc)'''     